# 메모리 기능을 갖춘 에이전트 생성하기

안녕하세요! 이번 튜토리얼에서는 **메모리 기능**을 갖춘 Amazon Bedrock 에이전트를 직접 만들어보겠습니다.

## 이 튜토리얼에서 배울 내용
- Amazon Bedrock 에이전트란 무엇인지
- 에이전트에 메모리 기능을 추가하는 방법
- 실제 여행 예약 시스템처럼 동작하는 AI 어시스턴트 만들기

## 우리가 만들 여행 도우미의 기능
1. **새로운 여행 예약하기** - 출발지, 목적지, 날짜, 교통수단 정보로 여행을 예약
2. **기존 예약 수정하기** - 이미 예약된 여행의 날짜를 변경
3. **예약 취소하기** - 더 이상 필요하지 않은 예약을 삭제
4. **메모리 기능** - 이전 대화 내용을 기억하여 더 자연스러운 대화 진행

**중요 안내**: 이 예제는 학습 목적으로 만들어진 것입니다. 실제 예약은 이루어지지 않으며, 시뮬레이션된 결과만 반환됩니다. 실제 서비스에서는 진짜 예약 시스템과 연결해야 합니다.

다음 구조도를 통해 우리가 구축할 시스템을 미리 살펴보세요:

![여행 도우미 에이전트](images/architecture.png)

## 1단계: 개발 환경 준비하기

시작하기 전에, 최신 기능을 사용하기 위해 필요한 패키지들을 업데이트하겠습니다.

**왜 업데이트가 필요한가요?**
- Amazon Bedrock의 에이전트 기능은 계속 발전하고 있습니다
- 최신 버전을 사용해야 메모리 기능 같은 새로운 기능을 사용할 수 있습니다
- 오래된 버전을 사용하면 예상치 못한 오류가 발생할 수 있습니다

In [ ]:
!python3 -m pip install --upgrade --force-reinstall -q boto3
!python3 -m pip install --upgrade --force-reinstall -q botocore
!python3 -m pip install --upgrade --force-reinstall -q awscli

설치가 완료되었으니, 이제 올바른 버전이 설치되었는지 확인해보겠습니다. 

**체크포인트**: boto3 버전이 1.34.139 이상이어야 메모리 기능을 사용할 수 있습니다. 만약 버전이 낮다면 위의 설치 명령을 다시 실행해주세요.

In [ ]:
import boto3
import botocore
import awscli
print(boto3.__version__)
print(botocore.__version__)
print(awscli.__version__)

## 2단계: 필요한 라이브러리 가져오기

이제 우리가 사용할 Python 라이브러리들을 가져오겠습니다. 각 라이브러리의 역할을 간단히 설명해드리겠습니다:

- **json**: 데이터를 JSON 형태로 변환하거나 처리할 때 사용
- **time**: 잠시 기다리거나 시간 관련 작업을 할 때 사용
- **zipfile**: 람다 함수 코드를 압축 파일로 만들 때 사용
- **uuid**: 고유한 ID를 생성할 때 사용 (예: 세션 ID, 예약 ID)
- **logging**: 프로그램 실행 과정을 기록하고 디버깅할 때 사용

In [ ]:
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging

In [ ]:
# 로깅 설정 - 프로그램 실행 과정을 추적하기 위해 설정합니다
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

print("로깅 설정이 완료되었습니다. 이제 프로그램 실행 과정을 상세히 확인할 수 있습니다.")

## 3단계: AWS 서비스 클라이언트 설정하기

이제 우리가 사용할 AWS 서비스들과 연결하기 위한 클라이언트들을 만들어보겠습니다.

**각 클라이언트의 역할**:
- **sts_client**: AWS 계정 정보 확인 (계정 ID 등)
- **iam_client**: 권한 관리 (역할, 정책 생성)
- **lambda_client**: 람다 함수 생성 및 관리
- **bedrock_agent_client**: Bedrock 에이전트 생성 및 관리
- **bedrock_agent_runtime_client**: 에이전트와 실제 대화하기 위해 사용

In [ ]:
# AWS 서비스 클라이언트 생성
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

print("모든 AWS 서비스 클라이언트가 성공적으로 생성되었습니다!")

## 4단계: 기본 정보 확인 및 설정값 정의하기

먼저 현재 사용 중인 AWS 계정과 리전 정보를 확인해보겠습니다. 이 정보는 리소스 이름을 고유하게 만들기 위해 사용됩니다.

In [ ]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

print(f"현재 사용 중인 AWS 리전: {region}")
print(f"현재 사용 중인 AWS 계정 ID: {account_id}")
print("기본 정보 확인 완료!")

region, account_id

In [ ]:
# 프로젝트 설정값들을 정의합니다
# 이 값들은 우리가 만들 모든 리소스의 이름을 결정합니다

suffix = f"{region}-{account_id}"  # 리소스 이름을 고유하게 만들기 위한 접미사

# 에이전트 관련 설정
agent_name = "travel-assistant-with-memory"
agent_description = "여행 예약을 도와주는 메모리 기능이 있는 AI 어시스턴트"

# 에이전트가 따를 지시사항 (에이전트의 역할과 행동 방식을 정의)
agent_instruction = """
당신은 고객의 여행 예약을 도와주는 여행사 상담원입니다.
여행 예약 생성, 수정, 삭제 기능을 제공할 수 있습니다.
항상 친절하고 도움이 되는 방식으로 고객을 응대해야 합니다.
"""

# IAM 역할 및 정책 이름들
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

# 액션 그룹 설정 (에이전트가 사용할 수 있는 기능들)
agent_action_group_name = "BookingManagerActionGroup"
agent_action_group_description = """
여행 예약을 관리하는 액션 그룹입니다. 
기차 및 항공 여행 예약의 생성, 수정, 삭제 기능을 제공합니다.
"""

# 기타 설정
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'
memory_time = 30  # 메모리를 30일간 보관

print(f"설정 완료")
print(f"   - 에이전트 이름: {agent_name}")
print(f"   - 메모리 보관 기간: {memory_time}일")
print(f"   - 람다 함수 이름: {lambda_function_name}")

## 5단계: AI 모델 선택하기

에이전트가 사용할 AI 모델을 선택합니다. **추론 프로필(Inference Profile)**을 사용하면 여러 리전에 걸쳐 요청을 효율적으로 분산시킬 수 있습니다.

**선택 가능한 모델들**:
- **Amazon Nova Pro**: 아마존에서 새로 출시한 강력한 모델 (추천)
- **Anthropic Claude 3.5 Sonnet v2**: 뛰어난 추론 능력을 가진 모델

아래에서 원하는 모델을 선택하거나 변경할 수 있습니다.

In [ ]:
# 사용할 AI 모델을 선택합니다
# 원하는 모델의 주석을 해제하고 다른 모델은 주석 처리하세요

inference_profile = "us.amazon.nova-pro-v1:0"          # Amazon Nova Pro (추천)
# inference_profile = "us.anthropic.claude-3-sonnet-20240229-v1:0"  # Claude 3.5 Sonnet

foundation_model = inference_profile[3:]  # 'us.' 접두사 제거

print(f"선택된 모델: {foundation_model}")
print(f"추론 프로필: {inference_profile}")

foundation_model

## 6단계: 비즈니스 로직을 처리할 람다 함수 만들기

이제 실제 여행 예약 기능을 처리할 **AWS Lambda 함수**를 만들어보겠습니다.

**람다 함수란?**
- 서버 관리 없이 코드를 실행할 수 있는 AWS 서비스입니다
- 에이전트가 특정 작업(예약, 수정, 삭제)을 요청할 때마다 이 함수가 실행됩니다

**우리 람다 함수가 제공하는 기능들**:
1. `book_trip` - 새로운 여행 예약
2. `update_existing_trip_dates` - 기존 예약의 날짜 변경
3. `delete_existing_trip_reservation` - 예약 취소

**중요**: 이 함수들은 실제로는 시뮬레이션만 하고, 진짜 예약은 하지 않습니다. 실제 서비스에서는 데이터베이스나 외부 API와 연결해야 합니다.

In [ ]:
%%writefile lambda_function.py
import json
import uuid
import boto3

def get_named_parameter(event, name):
    """
    에이전트로부터 전달받은 매개변수에서 특정 이름의 값을 찾아 반환합니다.
    
    Args:
        event: 람다 함수가 받은 이벤트 데이터
        name: 찾고자 하는 매개변수 이름
    
    Returns:
        해당 매개변수의 값
    """
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def book_trip(origin, destination, start_date, end_date, transportation_mode):
    """
    새로운 여행을 예약하는 함수 (시뮬레이션)
    
    Args:
        origin (string): 출발지
        destination (string): 목적지  
        start_date (string): 출발 날짜
        end_date (string): 귀국 날짜
        transportation_mode (string): 교통수단 (TRAIN 또는 AIR)
    
    Returns:
        예약 성공 메시지와 예약 ID
    """
    # 고유한 예약 ID 생성 (실제로는 데이터베이스에서 관리해야 함)
    booking_id = str(uuid.uuid4())[:8]
    
    return f"여행 예약이 완료되었습니다!\n출발: {origin} ({start_date})\n도착: {destination} ({end_date})\n교통수단: {transportation_mode}\n예약번호: {booking_id}"

def update_existing_trip_dates(booking_id, new_start_date, new_end_date):
    """
    기존 예약의 날짜를 변경하는 함수 (시뮬레이션)
    
    Args:
        booking_id (string): 변경할 예약의 ID
        new_start_date (string): 새로운 출발 날짜
        new_end_date (string): 새로운 귀국 날짜
    
    Returns:
        날짜 변경 성공 메시지
    """
    return f"예약 {booking_id}의 날짜가 성공적으로 변경되었습니다.\n새로운 출발일: {new_start_date}\n새로운 귀국일: {new_end_date}"

def delete_existing_trip_reservation(booking_id):
    """
    기존 예약을 취소하는 함수 (시뮬레이션)
    
    Args:
        booking_id (string): 취소할 예약의 ID
    
    Returns:
        예약 취소 성공 메시지
    """
    return f"예약 {booking_id}이(가) 성공적으로 취소되었습니다."

def populate_function_response(event, response_body):
    """
    람다 함수의 응답을 Bedrock 에이전트가 이해할 수 있는 형태로 변환합니다.
    """
    return {
        'response': {
            'actionGroup': event['actionGroup'], 
            'function': event['function'],
            'functionResponse': {
                'responseBody': {
                    'TEXT': {
                        'body': response_body
                    }
                }
            }
        }
    }


def lambda_handler(event, context):
    """
    람다 함수의 메인 핸들러
    Bedrock 에이전트에서 호출되면 실행되는 함수입니다.
    """
    # 에이전트에서 전달받은 정보들을 추출
    actionGroup = event.get('actionGroup', '')
    function = event.get('function', '')  # 실행할 함수 이름
    parameters = event.get('parameters', [])  # 함수에 전달할 매개변수들

    # 요청된 함수에 따라 적절한 처리 수행
    if function == 'update_existing_trip_dates':
        # 예약 날짜 변경 요청 처리
        booking_id = get_named_parameter(event, "booking_id")
        new_start_date = get_named_parameter(event, "new_start_date")
        new_end_date = get_named_parameter(event, "new_end_date")
        
        if booking_id and new_start_date and new_end_date:
            response = str(update_existing_trip_dates(booking_id, new_start_date, new_end_date))
            result = json.dumps(response)
        else:
            result = '필수 정보가 누락되었습니다: 예약번호, 새로운 출발일, 새로운 귀국일'

    elif function == 'book_trip':
        # 새 여행 예약 요청 처리
        origin = get_named_parameter(event, "origin")
        destination = get_named_parameter(event, "destination")
        start_date = get_named_parameter(event, "start_date")
        end_date = get_named_parameter(event, "end_date")
        transportation_mode = get_named_parameter(event, "transportation_mode")

        if all([origin, destination, start_date, end_date, transportation_mode]):
            response = str(book_trip(origin, destination, start_date, end_date, transportation_mode))
            result = json.dumps(response) 
        else:
            result = '필수 정보가 누락되었습니다: 출발지, 목적지, 출발일, 귀국일, 교통수단'
            
    elif function == 'delete_existing_trip_reservation':
        # 예약 취소 요청 처리
        booking_id = get_named_parameter(event, "booking_id")
        
        if booking_id:
            response = str(delete_existing_trip_reservation(booking_id))
            result = json.dumps(response)
        else:
            result = '예약번호가 필요합니다'
    else:
        result = '알 수 없는 기능 요청입니다'
    
    # 결과를 에이전트가 이해할 수 있는 형태로 변환하여 반환
    action_response = populate_function_response(event, result)
    print(action_response)  # 디버깅용 로그
    return action_response

## 7단계: 람다 함수 실행을 위한 권한 설정하기

람다 함수가 제대로 작동하려면 적절한 **IAM 역할**과 **권한**이 필요합니다.

**IAM 역할이란?**
- AWS 서비스가 다른 AWS 서비스에 접근할 때 사용하는 권한 집합입니다
- 마치 "출입증"과 같은 역할을 합니다

**여기서 설정하는 권한**:
- 람다 함수가 로그를 CloudWatch에 기록할 수 있는 권한
- 기본적인 람다 함수 실행 권한

In [ ]:
# 람다 함수를 위한 IAM 역할을 생성합니다
try:
    # 람다 서비스가 이 역할을 사용할 수 있도록 하는 정책
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"  # 람다 서비스가 이 역할을 사용할 수 있음
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )
    print("새로운 람다 IAM 역할을 생성했습니다.")

    # 역할이 완전히 생성될 때까지 잠시 대기
    time.sleep(10)
    
except Exception as e:
    # 이미 역할이 존재하는 경우 기존 역할을 사용
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)
    print("기존 람다 IAM 역할을 사용합니다.")

# 람다 함수가 CloudWatch에 로그를 기록할 수 있는 기본 권한 추가
iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

print("람다 함수 권한 설정이 완료되었습니다!")

## 8단계: 람다 함수 배포하기

이제 앞에서 작성한 Python 코드를 실제 람다 함수로 배포해보겠습니다.

**과정 설명**:
1. Python 파일을 ZIP 압축 파일로 만듭니다 (람다는 ZIP 파일로 코드를 받습니다)
2. 압축된 코드를 AWS 람다 서비스에 업로드합니다
3. 람다 함수가 생성되고 사용할 준비가 완료됩니다

In [ ]:
# 1. 람다 함수 코드를 ZIP 파일로 압축
print("📦 람다 함수 코드를 압축 중...")
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.close()
zip_content = s.getvalue()
print("코드 압축 완료!")

# 2. 압축된 코드로 람다 함수 생성
print("람다 함수를 AWS에 배포 중...")
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',        # Python 3.12 런타임 사용
    Timeout=180,                 # 최대 3분 실행 시간
    Role=lambda_iam_role['Role']['Arn'],  # 앞에서 만든 IAM 역할 사용
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'  # 실행할 함수 지정
)

print("람다 함수 배포 완료")
print(f"   함수 이름: {lambda_function_name}")
print(f"   함수 ARN: {lambda_function['FunctionArn']}")

lambda_function

## 9단계: Bedrock 에이전트 생성하기

이제 우리의 핵심인 **Bedrock 에이전트**를 만들어보겠습니다!

**에이전트를 만들기 위해 필요한 것들**:
1. **AI 모델 사용 권한** - 에이전트가 선택한 AI 모델을 사용할 수 있는 권한
2. **IAM 역할** - AWS 서비스들과 상호작용할 수 있는 권한
3. **메모리 기능** - 대화 내용을 기억할 수 있는 설정

먼저 에이전트가 우리가 선택한 AI 모델(Nova Pro 또는 Claude)을 사용할 수 있도록 권한을 설정하겠습니다.

In [ ]:
# 에이전트가 AI 모델을 사용할 수 있도록 하는 권한 정책을 만듭니다
print("에이전트 권한 정책 생성 중...")

bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",  # AI 모델 호출 권한
            "Resource": [
                f"arn:aws:bedrock:*::foundation-model/{foundation_model}",
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        },
        {
            "Sid": "AmazonBedrockAgentBedrockGetInferenceProfile", 
            "Effect": "Allow",
            "Action": [
                "bedrock:GetInferenceProfile",      # 추론 프로필 정보 조회
                "bedrock:ListInferenceProfiles",    # 사용 가능한 추론 프로필 목록 조회
                "bedrock:UseInferenceProfile"       # 추론 프로필 사용 권한
            ],
            "Resource": "*"
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

# 정책을 AWS에 생성
agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

print("에이전트 권한 정책 생성 완료")
print(f"   정책 이름: {agent_bedrock_allow_policy_name}")

agent_bedrock_policy

In [ ]:
# 에이전트를 위한 IAM 역할을 생성하고 권한 정책을 연결합니다
print("에이전트 IAM 역할 생성 중...")

# Bedrock 서비스가 이 역할을 사용할 수 있도록 하는 정책
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"  # Bedrock 서비스가 이 역할을 사용할 수 있음
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)

# IAM 역할 생성
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

print("에이전트 IAM 역할 생성 완료")

# 역할이 완전히 생성될 때까지 잠시 대기 (AWS에서 리소스가 준비되는 시간 필요)
print("역할 생성 완료를 위해 10초 대기 중...")
time.sleep(10)

# 앞에서 만든 권한 정책을 이 역할에 연결
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

print("에이전트에 AI 모델 사용 권한 부여 완료")
print(f"   역할 이름: {agent_role_name}")

agent_role

### 메모리 기능을 포함한 에이전트 생성하기

이제 드디어 **메모리 기능이 있는 에이전트**를 만들 시간입니다!

**메모리 기능이란?**
- 에이전트가 이전 대화 내용을 기억할 수 있는 능력입니다
- 예를 들어, 첫 번째 대화에서 "제 이름은 김민수입니다"라고 말하면, 나중에 "안녕하세요, 김민수님!"이라고 인사할 수 있습니다

**우리가 설정하는 메모리 기능**:
- **종류**: `SESSION_SUMMARY` (세션 요약 메모리)
- **보관 기간**: 30일
- **작동 방식**: 각 대화 세션이 끝날 때마다 중요한 내용을 요약해서 저장

**세션 요약 메모리의 장점**:
- 긴 대화 내용을 효율적으로 압축하여 저장
- 다음 대화에서 이전 맥락을 이해하고 더 자연스러운 응답 제공
- 메모리 ID를 사용하여 특정 사용자의 대화 기록을 구분해서 관리

In [ ]:
# 모든 준비가 완료되었으니 에이전트를 생성합니다!
print("메모리 기능을 갖춘 Bedrock 에이전트 생성 중...")

# 에이전트 역할이 완전히 준비될 때까지 추가 대기
print("에이전트 역할 준비 완료를 위해 30초 대기 중...")
time.sleep(30)

response = bedrock_agent_client.create_agent(
    agentName=agent_name,                           # 에이전트 이름
    agentResourceRoleArn=agent_role['Role']['Arn'], # 에이전트가 사용할 IAM 역할
    description=agent_description,                   # 에이전트 설명
    idleSessionTTLInSeconds=1800,                   # 비활성 세션 유지 시간 (30분)
    foundationModel=inference_profile,               # 사용할 AI 모델
    instruction=agent_instruction,                   # 에이전트 행동 지침
    
    # 메모리 기능 설정 - 여기가 핵심입니다
    memoryConfiguration={
        "enabledMemoryTypes": ["SESSION_SUMMARY"],   # 세션 요약 메모리 활성화
        "storageDays": 30                           # 30일간 메모리 보관
    }
)

print("에이전트 생성 완료")
print(f"   에이전트 이름: {agent_name}")
print(f"   메모리 기능: 활성화 (30일간 보관)")
print(f"   사용 모델: {foundation_model}")

response

에이전트가 성공적으로 생성되었습니다! 이제 에이전트 ID를 저장해두겠습니다. 이 ID는 앞으로 에이전트와 상호작용할 때 필요합니다.

In [ ]:
agent_id = response['agent']['agentId']
print(f"에이전트 ID: {agent_id}")
print("에이전트 ID가 저장되었습니다!")

agent_id

## 10단계: 에이전트에 실제 기능 추가하기 (액션 그룹)

에이전트를 만들었지만, 아직 여행 예약 기능을 할 수 없습니다. 이제 **액션 그룹(Action Group)**을 만들어서 에이전트가 실제로 예약, 수정, 취소 기능을 할 수 있도록 해보겠습니다.

**액션 그룹이란?**
- 에이전트가 수행할 수 있는 구체적인 기능들의 모음입니다
- 앞에서 만든 람다 함수와 연결되어 실제 작업을 처리합니다
- 에이전트가 "언제 어떤 기능을 사용해야 하는지" 알 수 있도록 각 기능을 자세히 설명합니다

**우리가 정의할 기능들**:
1. **book_trip**: 새로운 여행 예약
2. **update_existing_trip_dates**: 기존 예약 날짜 변경  
3. **delete_existing_trip_reservation**: 예약 취소

**Function Schema 방식 사용**:
이 예제에서는 `functionSchema`를 사용해서 기능을 정의합니다. 각 기능마다 이름, 설명, 필요한 매개변수들을 명시해야 합니다. 이렇게 하면 에이전트가 사용자의 요청을 이해하고 적절한 기능을 선택할 수 있습니다.

In [ ]:
# 에이전트가 사용할 수 있는 기능들을 정의합니다
print("에이전트 기능 정의 중...")

agent_functions = [
    {
        'name': 'book_trip',
        'description': '고객을 위해 새로운 여행을 예약하는 기능입니다. 출발지, 목적지, 날짜, 교통수단이 모두 필요합니다.',
        'parameters': {
            "origin": {
                "description": "여행 출발 도시 (예: 서울, 부산, 제주도)",
                "required": True,
                "type": "string"
            },
            "destination": {
                "description": "여행 목적지 도시 (예: 도쿄, 파리, 뉴욕)",
                "required": True,
                "type": "string"
            },
            "start_date": {
                "description": "여행 시작 날짜 (형식: YYYY-MM-DD, 예: 2024-07-15)",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "여행 종료 날짜 (형식: YYYY-MM-DD, 예: 2024-07-20)",
                "required": True,
                "type": "string"
            },
            "transportation_mode": {
                "description": "교통수단을 선택하세요. TRAIN(기차) 또는 AIR(비행기) 중 하나",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'update_existing_trip_dates',
        'description': '이미 예약된 여행의 출발일이나 귀국일을 변경하는 기능입니다. 예약번호와 새로운 날짜들이 필요합니다.',
        'parameters': {
            "booking_id": {
                "description": "변경할 예약의 고유 번호 (예약 시 제공받은 ID)",
                "required": True,
                "type": "integer"
            },
            "new_start_date": {
                "description": "새로운 여행 시작 날짜 (형식: YYYY-MM-DD)",
                "required": True,
                "type": "string"
            },
            "new_end_date": {
                "description": "새로운 여행 종료 날짜 (형식: YYYY-MM-DD)",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'delete_existing_trip_reservation',
        'description': '기존 여행 예약을 완전히 취소하는 기능입니다. 예약번호만 있으면 됩니다.',
        'parameters': {
            "booking_id": {
                "description": "취소할 예약의 고유 번호 (예약 시 제공받은 ID)",
                "required": True,
                "type": "integer"
            }
        }
    }
]

print("에이전트 기능 정의 완료!")
print(f"   총 {len(agent_functions)}개의 기능이 정의되었습니다:")
for i, func in enumerate(agent_functions, 1):
    print(f"   {i}. {func['name']}: {func['description'][:50]}...")

agent_functions

In [ ]:
# 이제 액션 그룹을 생성하여 에이전트와 람다 함수를 연결합니다
print("액션 그룹 생성 중...")

# 에이전트가 완전히 생성될 때까지 잠시 대기
print("에이전트 생성 완료를 위해 30초 대기 중...")
time.sleep(30)

# 액션 그룹 생성 - 에이전트와 람다 함수를 연결하는 다리 역할
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',  # 아직 개발 중인 버전이므로 DRAFT 사용
    
    # 람다 함수 연결 정보
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']  # 앞에서 만든 람다 함수의 ARN
    },
    
    actionGroupName=agent_action_group_name,
    description=agent_action_group_description,
    
    # 에이전트가 사용할 수 있는 기능들 정의
    functionSchema={
        'functions': agent_functions
    }
)

print("액션 그룹 생성 완료")
print(f"   액션 그룹 이름: {agent_action_group_name}")
print(f"   연결된 람다 함수: {lambda_function_name}")
print(f"   사용 가능한 기능: {len(agent_functions)}개")

agent_action_group_response

In [ ]:
agent_action_group_response

## 11단계: 에이전트가 람다 함수를 호출할 수 있도록 권한 부여하기

액션 그룹을 만들었지만, 아직 에이전트가 람다 함수를 실제로 호출할 수는 없습니다. 

**왜 추가 권한이 필요한가요?**
- AWS에서는 보안을 위해 기본적으로 모든 접근을 차단합니다
- 에이전트가 람다 함수를 호출하려면 명시적으로 "허용" 권한을 설정해야 합니다
- 이는 **리소스 기반 정책(Resource-based Policy)**을 통해 설정됩니다

**리소스 기반 정책이란?**
- 특정 리소스(여기서는 람다 함수)에 "누가 접근할 수 있는지"를 정의하는 규칙입니다
- "우리가 만든 Bedrock 에이전트만 이 람다 함수를 호출할 수 있다"라고 설정하는 것입니다

In [ ]:
# 람다 함수에 리소스 기반 정책을 추가하여 에이전트의 호출을 허용합니다
print("람다 함수 호출 권한 설정 중...")

response = lambda_client.add_permission(
    FunctionName=lambda_function_name,           # 권한을 설정할 람다 함수
    StatementId='allow_bedrock',                 # 이 권한 규칙의 고유 ID
    Action='lambda:InvokeFunction',              # 허용할 작업: 함수 호출
    Principal='bedrock.amazonaws.com',           # 허용할 서비스: Bedrock 서비스
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}"  # 특정 에이전트만 허용
)

print("에이전트의 람다 함수 호출 권한 설정 완료")
print(f"   허용된 에이전트 ID: {agent_id}")
print(f"   허용된 작업: 람다 함수 호출")

response

In [ ]:
response

## 12단계: 에이전트 준비하기 (배포 가능한 상태로 만들기)

모든 구성 요소를 만들었으니, 이제 에이전트를 **사용할 수 있는 상태**로 준비해야 합니다.

**"Prepare Agent"가 무엇인가요?**
- 지금까지 만든 에이전트는 "구성 중인 상태"입니다
- 실제로 대화를 하려면 모든 설정을 확정하고 "준비 완료" 상태로 만들어야 합니다
- 이 과정에서 AWS가 모든 구성 요소가 올바르게 연결되어 있는지 검증합니다

**DRAFT 버전이란?**
- 개발 및 테스트 용도로 사용하는 임시 버전입니다
- 언제든지 수정하고 다시 준비할 수 있습니다
- 실제 서비스에서는 정식 버전(Version)과 별칭(Alias)을 만들어 사용합니다

In [ ]:
print("에이전트를 사용 가능한 상태로 준비 중...")

response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)

print("에이전트 준비 완료")
print("   에이전트가 이제 대화할 준비가 되었습니다!")
print("   모든 구성요소가 올바르게 연결되었는지 AWS에서 검증을 완료했습니다.")

print(response)

In [ ]:
# 에이전트 준비가 완전히 완료될 때까지 잠시 대기
print("에이전트 준비 완료를 위해 30초 대기 중...")
time.sleep(30)

# 테스트용 에이전트 별칭 ID 설정
# TSTALIASID는 개발/테스트 단계에서 사용하는 기본 별칭입니다
agent_alias_id = "TSTALIASID"

print(f"에이전트 별칭 ID 설정 완료: {agent_alias_id}")
print("   이제 에이전트와 대화할 수 있습니다!")

# 메모리 기능을 갖춘 에이전트 호출

이제 `memoryId`를 전달하여 에이전트를 호출하고 가장 일반적인 메모리 API 처리 방법을 살펴보겠습니다.

먼저 에이전트를 호출하는 헬퍼 함수와 에이전트 이름을 통해 에이전트 ID를 가져오는 헬퍼 함수를 생성합니다.

`invoke_agent_helper` 함수는 사용자가 `session_id`와 함께 에이전트에 `query`를 보낼 수 있도록 합니다. 세션은 사용자가 에이전트와 나누는 앞뒤 대화의 턴을 정의합니다. 에이전트는 세션 내에서 전체 컨텍스트를 기억할 수 있습니다. 사용자가 세션을 종료하면 이 컨텍스트는 제거됩니다.

사용자는 `enable_trace` 불린 변수를 사용하여 추적 활성화 여부를 결정할 수 있고, `session_state` 변수를 통해 세션 상태를 딕셔너리로 전달할 수 있습니다.

새 `session_id`가 제공되면 에이전트는 이전 컨텍스트 없이 새 대화를 생성합니다. 동일한 `session_id`를 재사용하면 해당 세션과 관련된 대화 컨텍스트를 에이전트가 알고 있습니다.

`enable_trace`가 `True`로 설정되면 에이전트의 각 응답에는 에이전트가 조율하는 단계를 자세히 설명하는 *추적*이 함께 제공됩니다. 이를 통해 대화의 해당 시점에서 최종 응답으로 이어진 에이전트의 추론(연쇄 사고 프롬프팅을 통한)을 따라갈 수 있습니다.

메모리 기능을 처리하기 위해 `memory_id` 매개변수가 사용됩니다. 세션이 종료되면 `memory_id`의 일부로 새 세션 ID로 내용을 요약합니다.

마지막으로 `session_state` 매개변수를 사용하여 세션 컨텍스트를 전달할 수도 있습니다. 세션 상태를 사용하면 에이전트와 다음 정보를 공유할 수 있습니다:
- **`sessionAttributes`**: 사용자와 에이전트 간의 세션에서 지속되는 속성입니다. 동일한 session_id를 가진 모든 invokeAgent 호출은 동일한 세션에 속하며, 세션 시간 제한을 초과하지 않고 사용자가 세션을 종료하지 않는 한 sessionAttributes를 공유합니다. sessionAttributes는 람다 함수에서 사용할 수 있지만 에이전트 프롬프트에 추가되지 **않습니다**. 따라서 람다 함수가 처리할 수 있는 경우에만 세션 속성을 사용할 수 있습니다. 세션 속성 사용에 대한 더 많은 예제는 [여기](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-and-function-calling/bedrock-agents/features-examples/06-prompt-and-session-attributes)에서 찾을 수 있습니다. 람다 함수 통합을 사용하여 특정 API에 대한 세밀한 액세스 제어를 구현하는 것도 좋은 패턴입니다. 예제는 [여기](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-and-function-calling/bedrock-agents/features-examples/09-fine-grained-access-permissions)에서 확인할 수 있습니다.
- **`promptSessionAttributes`**: 단일 invokeAgent 호출에서 지속되는 속성입니다. 프롬프트 속성은 프롬프트와 람다 함수에 추가됩니다. 오케스트레이션 기본 프롬프트를 편집할 때 `$prompt_session_attributes$` 자리 표시자를 사용할 수도 있습니다.
- **`invocationId`**: InvokeAgent 응답의 returnControl 필드에 있는 [ReturnControlPayload](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_ReturnControlPayload.html) 객체에서 에이전트가 반환한 ID입니다. 이 필드는 Return of Control 호출의 답변을 전달하는 경우 필요합니다. 사용 방법에 대한 예제는 [여기](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-and-function-calling/bedrock-agents/features-examples/03-create-agent-with-return-of-control)에서 확인할 수 있습니다.
- **`returnControlInvocationResults`**: Amazon Bedrock 에이전트 외부에서 액션을 호출하여 얻은 결과입니다. 이 필드는 Return of Control 호출의 답변을 전달하는 경우 필요합니다. 사용 방법에 대한 예제는 [여기](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-and-function-calling/bedrock-agents/features-examples/03-create-agent-with-return-of-control)에서 확인할 수 있습니다.

또한 `TSTALIASID`로 설정된 테스트 `agent_alias_id`를 사용합니다. 이는 개발 중인 에이전트를 테스트하는 데 사용할 수 있는 기본값입니다. [에이전트를 배포](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-deploy.html)하여 에이전트의 새 버전을 생성하고 새 에이전트 별칭 ID를 가질 수도 있습니다.

In [ ]:
def invoke_agent_helper(
    query, session_id, agent_id, alias_id, enable_trace=False, memory_id=None, session_state=None, end_session=False
):
    """
    에이전트와 대화하기 위한 헬퍼 함수
    
    매개변수:
        query (str): 에이전트에게 보낼 질문이나 요청
        session_id (str): 대화 세션의 고유 ID
        agent_id (str): 호출할 에이전트의 ID
        alias_id (str): 에이전트 별칭 ID (보통 'TSTALIASID')
        enable_trace (bool): 에이전트의 추론 과정을 볼지 여부 (기본값: False)
        memory_id (str): 메모리 기능을 사용할 때의 메모리 ID
        session_state (dict): 세션 상태 정보 (선택사항)
        end_session (bool): 세션을 종료할지 여부 (기본값: False)
    
    반환값:
        str: 에이전트의 응답 텍스트
    """
    if not session_state:
        session_state = {}

    # 에이전트 API 호출
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,                     # 사용자 질문
        agentId=agent_id,                   # 에이전트 ID
        agentAliasId=alias_id,              # 에이전트 별칭 ID
        sessionId=session_id,               # 세션 ID
        enableTrace=enable_trace,           # 추적 기능 활성화 여부
        endSession=end_session,             # 세션 종료 여부
        memoryId=memory_id,                 # 메모리 ID
        sessionState=session_state          # 세션 상태
    )

    if enable_trace:
        logger.info(pprint.pprint(agent_response))

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"최종 답변 ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer
                # 요청이 성공적으로 완료되었음을 나타내는 종료 이벤트
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("예상치 못한 이벤트가 발생했습니다.", event)
    except Exception as e:
        raise Exception("예상치 못한 이벤트가 발생했습니다.", e)

### 빈 메모리로 대화 시작하기

이제 에이전트와 실제로 대화를 시작해보겠습니다! 첫 번째 대화에서는 메모리가 비어있는 상태에서 시작됩니다.

**대화 시나리오**:
사용자 Anna가 보스턴에서 뉴욕까지의 항공편 여행을 예약하려고 합니다. 에이전트가 어떻게 응답하는지 확인해보겠습니다.

In [ ]:
# 대화를 위한 고유한 세션 ID 생성
session_id: str = str(uuid.uuid1())  # 새로운 세션마다 고유한 ID
memory_id: str = 'TST_MEM_ID'        # 테스트용 메모리 ID
enable_trace: bool = False           # 추적 기능 비활성화 (깔끔한 출력을 위해)
end_session: bool = False           # 아직 세션을 종료하지 않음

# Anna의 첫 번째 요청: 여행 예약
query = "안녕하세요, 제 이름은 Anna입니다. 보스턴에서 뉴욕까지 2024년 7월 11일 출발해서 7월 22일에 돌아오는 항공편을 예약하고 싶습니다."

print("🎯 사용자 요청:")
print(f"   \"{query}\"\n")

print("🤖 에이전트 응답:")
response = invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=enable_trace, memory_id=memory_id)
print(response)

훌륭합니다! 에이전트가 여행 예약을 성공적으로 처리하고 예약 ID까지 제공했습니다. 

이제 Anna가 에이전트의 도움에 감사 인사를 해보겠습니다. 에이전트가 어떻게 응답하는지 확인해보세요.

In [ ]:
# Anna의 감사 인사
query = "감사합니다!"

print("🎯 사용자 요청:")
print(f"   \"{query}\"\n")

print("🤖 에이전트 응답:")
response = invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=enable_trace, memory_id=memory_id)
print(response)

이제 첫 번째 세션을 종료해보겠습니다. `end_session=True`로 설정하면 에이전트가 현재 세션을 마무리하고, 이 대화 내용을 요약해서 메모리에 저장합니다.

**중요**: 세션이 종료되면 에이전트는 이 대화에서 중요한 정보들(Anna의 이름, 예약 정보 등)을 메모리에 저장합니다.

In [ ]:
# 세션 종료
query = "대화를 마치겠습니다"

print("🎯 사용자 요청:")
print(f"   \"{query}\"\n")

print("🤖 에이전트 응답:")
response = invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=enable_trace, memory_id=memory_id, end_session=True)
print(response)

print("\n✅ 세션이 종료되었습니다!")
print("   이제 에이전트가 이 대화 내용을 메모리에 저장하기 시작합니다.")

### 메모리 생성 대기하기

에이전트가 세션 내용을 메모리로 변환하는 데는 시간이 걸립니다. 아래 함수는 메모리가 생성될 때까지 기다리는 헬퍼 함수입니다.

In [ ]:
def wait_memory_creation(agent_id, agent_alias_id, memory_id):
    """
    메모리가 생성될 때까지 대기하는 함수
    
    매개변수:
        agent_id (str): 에이전트 ID
        agent_alias_id (str): 에이전트 별칭 ID
        memory_id (str): 확인할 메모리 ID
    
    반환값:
        tuple: (메모리 생성 소요 시간, 메모리 내용)
    """
    print("📝 메모리 생성을 기다리는 중...")
    start_memory = time.time()
    memory_content = None
    
    if memory_id is not None:
        while not memory_content:
            time.sleep(5)  # 5초마다 확인
            print("   메모리 상태 확인 중...")
            try:
                memory_content = bedrock_agent_runtime_client.get_agent_memory(
                    agentAliasId=agent_alias_id,
                    agentId=agent_id,
                    memoryId=memory_id,
                    memoryType='SESSION_SUMMARY'
                )['memoryContents']
            except Exception as e:
                # 메모리가 아직 생성되지 않았을 수 있음
                continue
                
        end_memory = time.time()
        memory_creation_time = (end_memory - start_memory)
        print(f"✅ 메모리 생성 완료! (소요 시간: {memory_creation_time:.1f}초)")
    
    return memory_creation_time, memory_content

In [ ]:
# 메모리가 생성될 때까지 대기하고 내용 확인
memory_creation_time, memory_content = wait_memory_creation(agent_id, agent_alias_id, memory_id)

print(f"\n🧠 생성된 메모리 내용:")
print("=" * 50)
for content in memory_content:
    print(f"세션 ID: {content.get('sessionId', 'N/A')}")
    print(f"요약: {content.get('summary', 'N/A')}")
    print("-" * 30)

print(f"\n📊 메모리 통계:")
print(f"   생성 소요 시간: {memory_creation_time:.1f}초")
print(f"   저장된 세션 수: {len(memory_content)}개")

### 메모리 정보를 기반으로 에이전트 호출하기

이제 정말 흥미로운 부분입니다! 새로운 세션을 시작하지만, 같은 `memory_id`를 사용해서 에이전트가 이전 대화 내용을 기억할 수 있는지 확인해보겠습니다.

**테스트 시나리오**: Anna가 돌아와서 여행 날짜를 변경하고 싶어합니다. 에이전트가 Anna를 기억하고 이전 예약 정보를 알고 있는지 확인해보겠습니다.

In [ ]:
# 새로운 세션 ID 생성 (새로운 대화 시작)
session_id: str = str(uuid.uuid1())

# Anna의 두 번째 요청: 여행 날짜 변경
# 주목: Anna는 자신의 이름이나 이전 예약 정보를 다시 말하지 않습니다!
query = "하루 더 머물러야 할 것 같아요. 대신 23일에 돌아올 수 있을까요?"

print("🔄 새로운 세션 시작 (같은 memory_id 사용)")
print(f"🎯 사용자 요청:")
print(f"   \"{query}\"\n")

print("🤖 에이전트 응답:")
response = invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=enable_trace, memory_id=memory_id)
print(response)

print("\n💡 주목할 점:")
print("   - 에이전트가 Anna를 기억하고 있는지 확인하세요")
print("   - 이전 예약 정보를 알고 있는지 확인하세요")
print("   - 어떤 구체적인 정보를 확인하려고 하는지 보세요")

훌륭합니다! 에이전트가 메모리를 기반으로 Anna를 기억하고 이전 예약 정보를 정확히 파악했습니다.

이제 Anna가 에이전트가 제공한 정보가 맞다고 확인해주어서, 에이전트가 실제로 여행 날짜 변경 기능을 실행할 수 있도록 해보겠습니다.

In [ ]:
# Anna의 확인 응답
query = "네, 맞습니다! 변경해주세요."

print("🎯 사용자 요청:")
print(f"   \"{query}\"\n")

print("🤖 에이전트 응답:")
response = invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=enable_trace, memory_id=memory_id)
print(response)

print("\n✅ 예약 변경 완료!")
print("   에이전트가 메모리 정보를 활용해서 날짜 변경을 성공적으로 처리했습니다.")

In [ ]:
# 두 번째 세션도 종료
query = "감사합니다. 대화를 마치겠습니다."

print("🎯 사용자 요청:")
print(f"   \"{query}\"\n")

print("🤖 에이전트 응답:")
response = invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=enable_trace, memory_id=memory_id, end_session=True)
print(response)

print("\n✅ 두 번째 세션도 종료되었습니다!")
print("   이제 이 대화 내용도 메모리에 추가로 저장됩니다.")

### 업데이트된 메모리 조회하기

이제 에이전트에 저장된 메모리를 다시 확인해보겠습니다. 두 번의 세션이 모두 메모리에 저장되어 있는지 확인할 수 있습니다.

**주목할 점**:
- 첫 번째 세션: 초기 예약 정보
- 두 번째 세션: 날짜 변경 정보
- 모든 정보가 하나의 `memory_id`에 누적되어 저장됩니다

In [ ]:
# 업데이트된 메모리 내용 확인
memory_creation_time, updated_memory_content = wait_memory_creation(agent_id, agent_alias_id, memory_id)

print(f"\n🧠 업데이트된 메모리 내용:")
print("=" * 60)
for i, content in enumerate(updated_memory_content, 1):
    print(f"세션 {i}:")
    print(f"  세션 ID: {content.get('sessionId', 'N/A')}")
    print(f"  요약: {content.get('summary', 'N/A')}")
    print("-" * 50)

print(f"\n📊 메모리 통계:")
print(f"   업데이트 소요 시간: {memory_creation_time:.1f}초")
print(f"   총 저장된 세션 수: {len(updated_memory_content)}개")

print(f"\n🔍 메모리 기능 검증 결과:")
print("   ✅ 여러 세션의 정보가 하나의 memory_id에 누적 저장됨")
print("   ✅ 에이전트가 이전 대화 맥락을 정확히 기억함") 
print("   ✅ 메모리 정보를 활용한 기능 실행 성공")

## 메모리 삭제하기

이제 테스트를 마무리하기 위해 생성된 메모리를 삭제해보겠습니다. 

**중요 주의사항**: 
- `delete_agent_memory` 함수를 호출하면 해당 `memory_id`의 **모든** 세션 정보가 삭제됩니다
- 실제 서비스에서는 사용자가 명시적으로 요청할 때만 메모리를 삭제해야 합니다
- 메모리 삭제는 되돌릴 수 없는 작업입니다

In [ ]:
# 메모리 삭제 실행
print(f"🗑️ 메모리 삭제 중... (Memory ID: {memory_id})")

response = bedrock_agent_runtime_client.delete_agent_memory(
    agentAliasId=agent_alias_id, 
    agentId=agent_id, 
    memoryId=memory_id
)

print("✅ 메모리 삭제 완료!")
print(f"   응답 상태: {response['ResponseMetadata']['HTTPStatusCode']}")

# 응답 내용 확인
print("\n📄 삭제 응답 내용:")
print(response)

### 메모리 삭제 확인

메모리가 정말로 삭제되었는지 확인해보겠습니다. `get_agent_memory` 메서드를 다시 호출해서 빈 결과가 나오는지 확인합니다.

In [ ]:
# 메모리 삭제 확인
print("🔍 메모리 삭제 확인 중...")

try:
    response = bedrock_agent_runtime_client.get_agent_memory(
        agentAliasId=agent_alias_id, 
        agentId=agent_id, 
        memoryId=memory_id, 
        memoryType="SESSION_SUMMARY"
    )
    memory_contents = response['memoryContents']
    
    if not memory_contents:
        print("✅ 메모리가 성공적으로 삭제되었습니다!")
        print("   저장된 세션 정보가 없습니다.")
    else:
        print(f"⚠️ 아직 {len(memory_contents)}개의 세션이 남아있습니다.")
        
except Exception as e:
    print(f"✅ 메모리가 완전히 삭제되었습니다!")
    print(f"   (에러 메시지: {str(e)})")

print(f"\n📊 최종 상태:")
print(f"   Memory ID: {memory_id}")
print(f"   상태: 삭제됨")

## 리소스 정리 (선택사항)

튜토리얼을 마친 후에는 생성된 AWS 리소스들을 정리할 수 있습니다. 

**⚠️ 주의사항**: 
- 아래 셀을 실행하면 생성했던 모든 리소스가 삭제됩니다
- 실제 프로젝트에서 사용 중인 리소스는 삭제하지 마세요
- 리소스 삭제는 되돌릴 수 없는 작업입니다

**정리될 리소스들**:
- Bedrock 에이전트 및 액션 그룹
- Lambda 함수
- IAM 역할 및 정책

리소스를 정리하려면 아래 셀의 주석을 해제하고 실행하세요.

In [ ]:
# 리소스 정리 (주석을 해제하여 실행)
"""
print("🧹 리소스 정리 시작...")

# 1. 액션 그룹 비활성화 및 삭제
print("1️⃣ 액션 그룹 삭제 중...")
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

# 액션 그룹 비활성화
response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId=action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

# 액션 그룹 삭제
action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId=action_group_id
)

# 2. 에이전트 삭제
print("2️⃣ 에이전트 삭제 중...")
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

# 3. Lambda 함수 삭제
print("3️⃣ Lambda 함수 삭제 중...")
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

# 4. IAM 역할에서 정책 분리
print("4️⃣ IAM 정책 분리 중...")
for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(
        RoleName=agent_role_name, 
        PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
    )
    
iam_client.detach_role_policy(
    RoleName=lambda_function_role, 
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

# 5. IAM 역할 삭제
print("5️⃣ IAM 역할 삭제 중...")
for role_name in [agent_role_name, lambda_function_role]:
    iam_client.delete_role(
        RoleName=role_name
    )

# 6. IAM 정책 삭제
print("6️⃣ IAM 정책 삭제 중...")
for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
    )

print("✅ 모든 리소스 정리 완료!")
print("   생성했던 모든 AWS 리소스가 성공적으로 삭제되었습니다.")
"""

print("💡 리소스 정리 코드가 준비되어 있습니다.")
print("   필요시 위의 주석을 해제하고 실행하세요.")

## 🎉 튜토리얼 완료!

축하합니다! **메모리 기능을 갖춘 Amazon Bedrock 에이전트**를 성공적으로 만들고 테스트해보았습니다.

## 📚 이 튜토리얼에서 배운 내용

### 1. **에이전트 구축 과정**
- AWS Lambda 함수로 비즈니스 로직 구현
- IAM 역할과 권한 설정
- Bedrock 에이전트 생성 및 구성
- 액션 그룹을 통한 기능 연결

### 2. **메모리 기능 활용**
- 세션 요약 메모리(`SESSION_SUMMARY`) 설정
- 다중 세션에 걸친 정보 누적 저장
- 메모리 기반 맥락 인식 대화
- 메모리 조회 및 관리

### 3. **실제 사용 시나리오**
- 사용자 정보 기억 (이름, 선호도 등)
- 이전 거래 내역 활용
- 맥락을 이해한 자연스러운 대화
- 개인화된 서비스 제공

## 🚀 다음 단계

### 실제 서비스 적용을 위한 개선사항
1. **데이터베이스 연동**: 시뮬레이션이 아닌 실제 데이터베이스 사용
2. **사용자 인증**: 각 사용자별로 고유한 memory_id 관리
3. **오류 처리**: 예외 상황에 대한 견고한 처리 로직
4. **보안 강화**: 민감한 정보 암호화 및 접근 제어
5. **성능 최적화**: 대용량 메모리 효율적 관리

### 추가 기능 실험
- **프롬프트 최적화**: 에이전트 응답 품질 향상
- **다양한 액션 그룹**: 더 복잡한 기능 추가
- **멀티모달 기능**: 이미지, 문서 처리 기능
- **실시간 통합**: 웹소켓이나 스트리밍 API 활용

## 💡 핵심 포인트

> **메모리 기능은 단순히 정보를 저장하는 것이 아니라, 사용자와의 지속적인 관계를 구축하는 핵심 기술입니다.**

이제 여러분만의 메모리 기능을 갖춘 AI 어시스턴트를 만들어보세요!

## 🔗 참고 자료
- [Amazon Bedrock Agents 공식 문서](https://docs.aws.amazon.com/bedrock/latest/userguide/agents.html)
- [Bedrock Agents 메모리 기능 가이드](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-memory.html)
- [AWS Lambda 개발자 가이드](https://docs.aws.amazon.com/lambda/latest/dg/)

---
**감사합니다! 🙏**